## Setup and configuration

1. Firstly, we'll import usefull packages.
2. Then, we'll load the data, before visualize and preprocess it.
3. We'll try a simple CNN model and then we will evaluate its performances.
4. And finally, we'll use techniques such as data augmentation, learning rate decay and dropout to increase our model's accuracy.

**Import Packages**

In [0]:
import numpy as np                          # linear algebra
import os                                   # used for loading the data
from sklearn.metrics import confusion_matrix# confusion matrix to carry out error analysis
import seaborn as sn                        # heatmap
from sklearn.utils import shuffle           # shuffle the data
import matplotlib.pyplot as plt             # 2D plotting library
import cv2                                  # image processing library
import tensorflow as tf                     # best library ever

In [0]:
# Here's our 6 categories that we have to classify.
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {'mountain': 0,
                    'street' : 1,
                    'glacier' : 2,
                    'buildings' : 3,
                    'sea' : 4,
                    'forest' : 5
                    }
nb_classes = 6